# Definitions
__Virus__ – A general term referring to a viral species (singular). It encompasses all genetic variants within that species.<br>
__Example__: Bunyamwera orthobunyavirus (a species within the Orthobunyavirus genus, Peribunyaviridae family).

__Virus strain__ – A genetically distinct variant within a species. Strains can arise due to mutation, reassortment, or adaptation to a new host or environment, but they still classified within the same viral species.<br>
__Example__: A strain of Bunyamwera orthobunyavirus that has adapted to a new mosquito host and differs slightly in its genome from the reference strain.

__Virus isolate__ – A specific sample of a virus obtained from a particular host, location, and time. Isolates are typically identified by accession numbers and may represent different virus strains. (TODO: is this a typo? should it be "same virus strains"?)<br>
__Example__: An isolate of Bunyamwera orthobunyavirus collected from a mosquito in Brazil in 2019, given an NCBI GenBank accession number like NC_038733.1 (for the medium RNA segment).<br>


# How These Concepts Relate to GCA/GCF Assemblies
__Same virus?__ → If two sequences belong to the same species (Bunyamwera orthobunyavirus), they are from the same virus in the broadest sense.

__Same virus strain?__ → If they share a nearly identical genome but come from different isolates, they are likely the same strain.

__Same virus isolate?__ → If two sequences originate from the same accession number (i.e., the same sample from the same host, location, and time), then they are the same isolate.


# Accession Number vs. Assembly ID

__Accession Number__ → A unique id for a specific genomic sequence submission in a public database (e.g., GenBank, RefSeq).<br>
Example: NC_038733.1 (a GenBank accession number for the medium RNA segment of Bunyamwera orthobunyavirus).<br>
Each segment (L, M, S) typically has its own accession number.

__Assembly ID__ → A unique id for a complete genome assembly (which may include multiple accession numbers, one per segment).<br>
Example: GCA_002831345.1 (a GenBank Assembly ID for a full genome).<br>
Example: GCF_002831345.1 (a RefSeq Assembly ID, a curated version of the same genome).

In [ ]:
import os
import json
import re
from collections import Counter
from pathlib import Path
from time import time
import numpy as np
import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# filepath = Path(__file__).parent # .py
filepath = Path(os.path.abspath('')) # .ipynb
print(filepath)

/nfs/lambda_stor_01/data/apartin/projects/cepi/viral-segmatch/notebooks


In [2]:
# task_name = 'Bunya-from-datasets'
task_name = 'bunya_processed'
data_dir = filepath / '../data'
raw_data_dir = data_dir / 'raw'
quality_gto_dir = raw_data_dir / 'Bunya-from-datasets/Quality_GTOs'

output_dir = data_dir / task_name
os.makedirs(output_dir, exist_ok=True)

print(raw_data_dir)
print(quality_gto_dir)
print(output_dir)

/nfs/lambda_stor_01/data/apartin/projects/cepi/viral-segmatch/notebooks/../data/raw
/nfs/lambda_stor_01/data/apartin/projects/cepi/viral-segmatch/notebooks/../data/raw/Bunya-from-datasets/Quality_GTOs
/nfs/lambda_stor_01/data/apartin/projects/cepi/viral-segmatch/notebooks/../data/bunya_processed


In [3]:
ex_file = 'GCF_031497195.1'
# ex_file = 'GCA_000851025.1'

# Utils

In [4]:
def extract_assembly_info(file_name):
    """ Extracts assembly prefix and assembly ID from a file name. """
    match = re.match(r'^(GCA|GCF)_(\d+\.\d+)', file_name)
    if match:
        return match.group(1), match.group(2)
    else:
        return None, None

# contig_quality files

In [5]:
# contig_quality
print(f"Total files: {len(sorted(quality_gto_dir.glob('*.contig_quality')))}")
cq = pd.read_csv(quality_gto_dir / f'{ex_file}.contig_quality', sep='\t')
cq

Total files: 1572


,Contig,Segment,Copy_Num,Len,Min_Len,Max_len,AMB_Bases,Frac_AMB,Exceptions
0,NC_086348.1,Large RNA Segment,1,6374,5764.0,10736.0,0,0,NaN
1,NC_086347.1,Medium RNA Segment,1,4310,NaN,NaN,0,0,NaN
2,NC_086346.1,Small RNA Segment,1,1863,1350.0,3435.0,0,0,NaN


# feature_quality files

In [6]:
# feature_quality
print(f"Total files: {len(sorted(quality_gto_dir.glob('*.feature_quality')))}")
fq = pd.read_csv(quality_gto_dir / f'{ex_file}.feature_quality', sep='\t')
fq

Total files: 1572


,ID,Function,Evaluated,Prot_Len,Min_Len,Max_Len,Copy_Num,Exp_Copy_Num,Exceptions
0,fig|426786.38.CDS.1,Nucleocapsid protein (N protein),1.0,245,205.0,540.0,1.0,1.0,NaN
1,fig|426786.38.CDS.2,Small Nonstructural Protein NSs (NSs Protein),NaN,226,NaN,NaN,NaN,NaN,NaN
2,fig|426786.38.CDS.3,Pre-glycoprotein polyprotein GP complex (GPC p...,NaN,1366,NaN,NaN,NaN,NaN,NaN
3,fig|426786.38.CDS.4,RNA-dependent RNA polymerase (L protein),1.0,2083,1719.0,3211.0,1.0,1.0,NaN
4,fig|426786.38.mat_peptide.1,Phenuiviridae mature nonstructural 78-kD protein,NaN,647,NaN,NaN,NaN,NaN,NaN
5,fig|426786.38.mat_peptide.2,Mature envelope glycoprotein Gn (Gn protein),NaN,512,NaN,NaN,NaN,NaN,NaN
6,fig|426786.38.mat_peptide.3,Mature envelope glycoprotein Gc (Gc protein),NaN,506,NaN,NaN,NaN,NaN,NaN


# qual.gto files

In [7]:
# qual.gto
print(f"Total files: {len(sorted(quality_gto_dir.glob('*.qual.gto')))}")
file_path = quality_gto_dir / f'{ex_file}.qual.gto'

# Load the GTP file
with open(file_path, 'r') as file:
    gto = json.load(file)

for k in sorted(gto.keys()):
    print(f'{k}: {type(gto[k])}')

Total files: 1572
analysis_events: <class 'list'>
close_genomes: <class 'list'>
contigs: <class 'list'>
domain: <class 'str'>
features: <class 'list'>
genetic_code: <class 'int'>
id: <class 'str'>
ncbi_taxonomy_id: <class 'int'>
quality: <class 'dict'>
scientific_name: <class 'str'>


In [8]:
# Explore 'features' in the GTO dict
print(f"Total 'features' items: {len(gto['features'])}") # that's the number rows in feature_quality file
for d in gto['features']:
    print(sorted(d.keys())) # print sorted keys

# create a list of sorted dicts
fea = []
for d in gto['features']:
    fea.append({k: d[k] for k in sorted(d.keys())})

Total 'features' items: 7
['annotations', 'family_assignments', 'feature_creation_event', 'function', 'id', 'location', 'protein_translation', 'type']
['annotations', 'family_assignments', 'feature_creation_event', 'function', 'id', 'location', 'protein_translation', 'type']
['annotations', 'family_assignments', 'feature_creation_event', 'function', 'id', 'location', 'protein_translation', 'type']
['annotations', 'family_assignments', 'feature_creation_event', 'function', 'id', 'location', 'protein_translation', 'type']
['annotations', 'family_assignments', 'feature_creation_event', 'function', 'id', 'location', 'protein_translation', 'type']
['annotations', 'family_assignments', 'feature_creation_event', 'function', 'id', 'location', 'protein_translation', 'type']
['annotations', 'family_assignments', 'feature_creation_event', 'function', 'id', 'location', 'protein_translation', 'type']


In [9]:
fea[0]

{'annotations': [['Add feature',
   'annotate_by_viral_pssm',
   1742556319.00236,
   '2B74E144-0647-11F0-A1B3-509768D45E13'],
  ['Set function to Nucleocapsid protein (N protein)',
   'annotate_by_viral_pssm',
   1742556319.00236,
   '2B74E144-0647-11F0-A1B3-509768D45E13']],
 'family_assignments': [['Phenuiviridae',
   'Phenuiviridae.N.3.pssm',
   'Nucleocapsid protein (N protein)',
   'annotate_by_viral_pssm']],
 'feature_creation_event': '2B74E144-0647-11F0-A1B3-509768D45E13',
 'function': 'Nucleocapsid protein (N protein)',
 'id': 'fig|426786.38.CDS.1',
 'location': [['NC_086346.1', '70', '+', '738']],
 'protein_translation': 'MTDYADIAIAFAGEPINNAEVMGWVNEFAYEGFNAQRIIQLVQEKGPQTWQTDVKMMIVLALTRGNKPSKMIEKMSAEGKKKASRLITIYGLKSGNPGRDDLTLSRIAAAFAGWTCQALATLHPYLPVTGAAMDAISPGYPRAMMHPSFAGLIDNSIPEAYLQVVVDAHALYLLQFSRVINRNMRGQPKSVVVSSFLQPMNAAIVSGFISNDRRRKMLMAFGIVDQNGKPTAAVESAAKAFMTAV*',
 'type': 'CDS'}

In [10]:
for i, item in enumerate(fea):
    print(f"{item['id']},  {item['type']},  {item['function']}")

fig|426786.38.CDS.1,  CDS,  Nucleocapsid protein (N protein)
fig|426786.38.CDS.2,  CDS,  Small Nonstructural Protein NSs (NSs Protein)
fig|426786.38.CDS.3,  CDS,  Pre-glycoprotein polyprotein GP complex (GPC protein)
fig|426786.38.CDS.4,  CDS,  RNA-dependent RNA polymerase (L protein)
fig|426786.38.mat_peptide.1,  mat_peptide,  Phenuiviridae mature nonstructural 78-kD protein
fig|426786.38.mat_peptide.2,  mat_peptide,  Mature envelope glycoprotein Gn (Gn protein)
fig|426786.38.mat_peptide.3,  mat_peptide,  Mature envelope glycoprotein Gc (Gc protein)


# DNA/RNA (contigs) data

In [ ]:
def get_dna_data_from_gto(gto_file_path):
    """
    Extract dna/rna data and metadata from a GTO file and return it as a DataFrame.
    """
    # file_path = quality_gto_dir / 'GCF_031497195.1.qual.gto'

    ## Load the GTO file
    with open(gto_file_path, 'r') as file:
        gto = json.load(file)

    ## Extract data from 'contigs'
    # Each item in 'contigs' is a dict with keys: 'id', 'replicon_type', 'replicon_geometry', 'dna'
    # ctg is a list of dicts, each representing a 'contigs' item
    cng = []
    for d in gto['contigs']:
        cng.append({k: d[k] for k in sorted(d.keys())})

    # Aggregate the 'contigs' items into a DataFrame
    df = []
    # contigs_columns contains available data items from 'contigs' key
    contigs_columns = ['id', 'replicon_type', 'replicon_geometry', 'dna'] # 'id' is an Accession Number
    for i, item in enumerate(cng):
        df.append([item[f] if f in item else np.nan for f in contigs_columns])
    df = pd.DataFrame(df, columns=contigs_columns)
    df['length'] = df['dna'].apply(lambda x: len(x) if isinstance(x, str) else 0)

    ## Extract additional metadata from GTO file
    assembly_prefix, assembly_id = extract_assembly_info(gto_file_path.name)
    meta = {
        'file': gto_file_path.name,
        'assembly_prefix': assembly_prefix,
        'assembly_id': assembly_id,
        'ncbi_taxonomy_id': gto['ncbi_taxonomy_id'],
        'genetic_code': gto['genetic_code'],
        'scientific_name': gto['scientific_name'],
        'quality': gto['quality']['genome_quality'],
    }
    meta_df = pd.DataFrame([meta])
    df = pd.merge(meta_df, df, how='cross')

    return df

In [ ]:
# Aggregate dna data from all GTO files
print(f"Total .qual.gto files: {len(sorted(quality_gto_dir.glob('*.qual.gto')))}")

dna_data_fname = 'dna_data_all.tsv'

if not (output_dir / dna_data_fname).exists():
    print(f'Creating {output_dir / dna_data_fname}')
    dfs = []    
    for i, fpath in enumerate(sorted(quality_gto_dir.glob('*.gto'))):
        df = get_dna_data_from_gto(gto_file_path=fpath)
        dfs.append(df)

    dna_df = pd.concat(dfs, axis=0).reset_index(drop=True)
    dna_df.to_csv(output_dir / dna_data_fname, sep='\t', index=False)

else:
    print(f'Loading {output_dir / dna_data_fname}')
    dna_df = pd.read_csv(output_dir / dna_data_fname, sep='\t')

print(f'DNA all samples {dna_df.shape}')
print(dna_df.columns.tolist())

Total files: 1572
Creating /nfs/lambda_stor_01/data/apartin/projects/cepi/viral-segmatch/notebooks/../data/bunya_processed/dna_data_all.tsv
(4686, 12)
['file', 'assembly_prefix', 'assembly_id', 'ncbi_taxonomy_id', 'genetic_code', 'scientific_name', 'quality', 'id', 'replicon_type', 'replicon_geometry', 'dna', 'length']


In [ ]:
# Save samples that missing seq data
dna_df_no_seq = dna_df[dna_df['dna'].isna()]
print(f'DNA with missing sequence {dna_df_no_seq.shape}')
# dna_df_no_seq.to_csv(output_dir / 'missing_dna_data.tsv', sep='\t', index=False)
dna_df_no_seq.to_csv(output_dir / 'dna_data_no_seq.tsv', sep='\t', index=False)

(0, 12)


In [14]:
# df = dna_df.drop(columns=['dna', 'length'])
# df.to_csv(output_dir / 'dna_data_no_seq.tsv', sep='\t', index=False)

### 1. Check Ambiguous DNA Data

In [ ]:
"""
Ambiguous DNA Data.
Remove sequences with too many unknown bases ('N' in DNA or 'X' in protein).
"""

def summarize_dna_qc(df, seq_col='dna'):
    """ Perform quality control (QC) on DNA sequences in a DataFrame. """
    df = df.copy()
    ambig_codes = set("NRYWSKMBDHV")  # IUPAC ambiguous nucleotide codes

    qc = []
    for idx, seq in df[seq_col].items():
        if not isinstance(seq, str) or len(seq) == 0:
            qc.append((0, 0.0, 0, 0.0))  # Handle empty or missing sequences
            continue

        seq_length = len(seq)
        ambig_count = sum(1 for base in seq if base in ambig_codes)
        ambig_frac = ambig_count / seq_length
        gc_content = (seq.count('G') + seq.count('C')) / seq_length
        
        qc.append((ambig_count, ambig_frac, seq_length, gc_content))

    columns = ['ambig_count', 'ambig_frac', 'length', 'gc_content']
    qc_df = pd.DataFrame(qc, columns=columns).reset_index(drop=True)
    df_combined = pd.concat([df, qc_df], axis=1)
    # return df, qc_df, df_combined
    return df_combined

dna_qc = summarize_dna_qc(dna_df, seq_col='dna')

# There is only one sample with ambiguous_frac of > 0.1.
# TODO! How should we filter this??
display(dna_qc[:3])
print(dna_qc['ambig_frac'].value_counts().sort_index(ascending=False))

,file,assembly_prefix,assembly_id,ncbi_taxonomy_id,genetic_code,scientific_name,quality,id,replicon_type,replicon_geometry,dna,length,ambig_count,ambig_frac,length,gc_content
0,GCA_000847345.1.qual.gto,GCA,000847345.1,11588,11,Rift Valley fever virus,Good,DQ375403.1,Large RNA Segment,linear,ACACAAAGGCGCCCAATCATGGATTCTATATTATCAAAACAGCTGG...,6404,0,0.0,6404,0.438944
1,GCA_000847345.1.qual.gto,GCA,000847345.1,11588,11,Rift Valley fever virus,Good,DQ380206.1,Medium RNA Segment,NaN,ACACAAAGACGGTGCATTAAATGTATGTTTTATTAACAATTCTAAT...,3885,0,0.0,3885,0.454569
2,GCA_000847345.1.qual.gto,GCA,000847345.1,11588,11,Rift Valley fever virus,Good,DQ380151.1,Small RNA Segment,linear,ACACAAAGACCCCCTAGTGCTTATCAAGTATATCATGGATTACTTT...,1690,0,0.0,1690,0.489349


ambig_frac
0.169765       1
0.033234       2
0.028332       1
0.023320       1
0.019280       1
            ... 
0.000138       2
0.000113       1
0.000083       1
0.000082       2
0.000000    4457
Name: count, Length: 147, dtype: int64


### 2. Check all accession numbers are unique.

In [ ]:
"""
Check that all accession numbers are unique.
"""
# All segment IDs are unique (no duplicates).
print('\nCheck that all accession numbers are unique.')
print(f"Total accession ids:        {dna_df['id'].count()}")
print(f"Total unique accession ids: {dna_df['id'].nunique()}")

Total ids:  4686
Unique ids: 4686


### 3. Check replicon types

In [ ]:
"""
Check replicon types.
"""
# Most segments follow the "[Large/Medium/Small] RNA Segment" naming convention.
# Some segments are labeled as "Segment One", "Segment Two", ...
# 311 entries have missing values
# TODO! 1) How can we standardize the "Segment [One/Two/...]" labels? 2) How can we handle the missing values?
display(dna_df['replicon_type'].value_counts(dropna=False))

replicon_type
Large RNA Segment     1503
Small RNA Segment     1371
Medium RNA Segment    1283
NaN                    311
Segment One             61
Segment Two             59
Segment Four            53
Segment Three           45
Name: count, dtype: int64

### 4. Check duplicates

In [ ]:
"""
Check how often each duplicated sequence appears across different files.
This step helps understand whether we should keep and remove certain duplicates.

===============
If 2 duplicates
===============

--------
Case 1:
--------
Sequecnes (same), Assembly IDs (same), Accession Numbers (same), Versions (different)

This is likely a technical duplicate (e.g., GCA/GCF versions of the same assembly for a given viral isolate). Some GTO files may contain both draft (GCA) and reference (GCF) assemblies for the same viral isolate, leading to identical sequences appearing in different files.
GCA (Genomic Contig Assembly): A draft genome assembly, typically submitted first.
GCF (Genomic Complete Assembly): A higher-quality, curated version of the same assembly.

Example:
file                        assembly_id     Accession Number    dna
GCA_002831345.1.qual.gto    002831345.1     NC_038733.1         AAAACAGTAGTGTACCG...
GCF_002831345.1.qual.gto    002831345.1     NC_038733.1         AAAACAGTAGTGTACCG...

Accession Number: NC_038733.1 (same RNA segment, same isolate)
File names: GCA vs. GCF (same assembly ID, different versions)

Conclusion:
Keep the GCF version (it's typically a higher-quality). TODO confirm with Jim

--------
Case 2:
--------
Sequences (same), Assembly IDs (same), Accession Numbers (different), Versions (ignore)

This means that the same genomic segment has been assigned different accession numbers within the same assembly. This can happen due to:
Reannotation: The sequence may have been re-submitted with updated metadata.
Redundant entries: Multiple GenBank submissions within the same assembly.

Example:
file                        assembly_id     Accession Number    dna
GCA_002831345.1.qual.gto    002831345.1     NC_038733.1         AAAACAGTAGTGTACCG...
GCA_002831345.1.qual.gto    002831345.1     OL123456.1          AAAACAGTAGTGTACCG...

Conclusion:
These are likely technical duplicates.
Keep only one — preferably the most recent or official accession number. TODO confirm with Jim

--------
Case 3:
--------
Sequences (same), Assembly IDs (different), Accession Numbers (different), Versions (ignore)

This suggests the same sequence appears in multiple genome assemblies, likely from different isolates.

Example:
file                        assembly_id     Accession Number    dna
GCA_002831345.1.qual.gto    002831345.1     NC_038733.1         AAAACAGTAGTGTACCG...
GCA_018595275.1.qual.gto    018595275.1     OL987654.1          AAAACAGTAGTGTACCG...

Conclusion:
If the assemblies are from different viral isolates, this reflects true biological conservation — keep both.
If they are somehow from the same isolate, consider keeping only the higher-quality version.


=========================
If more than 2 duplicates
=========================
Case 1:
TODO.

=========================
Result (ap):
=========================
Most duplicates appear in only 2 files.
We have 3 seqs that appear in 3 files, and 3 seqs that appear in 5 files.
num_files	count
2	        1592
3	        3 
5	        3
"""
dna_dups = dna_df[dna_df.duplicated(subset=['dna'], keep=False)].sort_values(['dna'])
dna_dups.to_csv(output_dir / 'dna_all_duplicates.tsv', sep='\t', index=False)

print(f"Duplicates on 'dna': {dna_dups.shape}")
print(dna_dups[:4])

# dna_dups.groupby('dna').agg(file=('file', 'unique')).reset_index().explode('file')

dup_counts = dna_dups[['dna', 'file']].groupby('dna')['file'].nunique().reset_index()
dup_counts['file'].value_counts().reset_index()

,file,count
0,2,1592
1,3,3
2,5,3


In [ ]:
# df.groupby('dna') retrieves the rows with the same dna sequence (i.e., dna sequence duplicates)
# grouped = dna_dups.groupby('dna')

# # Explore the first group
# for seq, group in grouped:
#     print(f"Sequence: {seq[:50]}")  # print a slice of seq
#     print("Group (rows with this seq):")
#     display(group)
#     break  # remove this to loop over more groups

In [ ]:
def classify_dup_groups(df):
    """
    Classifies duplicate groups based on the number of unique assembly IDs, accession numbers, and prefixes.
    """
    dup_info = []

    # df.groupby('dna') retrieves the rows with the same dna sequence (i.e., dna sequence duplicates)
    for seq, group in df.groupby('dna'):
        files = list(group['file'])               # GCA_018595275.1.qual.gto
        assembly_ids = list(group['assembly_id']) # 18595275.1
        accession_ids = list(group['id'])         # MW039262.1
        prefixes = list(group['assembly_prefix']) # GCA
        
        case = None
        if len(set(assembly_ids)) == 1 and len(set(accession_ids)) == 1 and set(prefixes) == {'GCA', 'GCF'}:
            # Sequecnes (same), Assembly IDs (same), Accession Numbers (same), Versions (different)
            case = 'Case 1'
        elif len(set(assembly_ids)) == 1 and len(set(accession_ids)) > 1:
            # Sequences (same), Assembly IDs (same), Accession Numbers (different), Versions (ignore)
            case = 'Case 2'
        elif len(set(assembly_ids)) > 1 and len(set(accession_ids)) > 1:
            # Sequences (same), Assembly IDs (different), Accession Numbers (different), Versions (ignore)
            case = 'Case 3'
        else:
            case = 'Other'

        dup_info.append({
            'dna': seq,
            'num_dups': len(group),
            'files': files,
            'assembly_ids': list(set(assembly_ids)),
            'accession_ids': list(set(accession_ids)),
            'prefixes': list(set(prefixes)),
            'case': case
        })

    return pd.DataFrame(dup_info)

In [22]:
"""
✅ Summary of Duplicate Classifications.
Case	Description	                                            Count	Suggested Action
Case 1	Same assembly_id, same accession_id, GCA & GCF versions	0	    ✅ Nothing to do
Case 2	Same assembly_id, different accession numbers	        1,524	⚠️ Technical duplicates — keep only one
Case 3	Different assembly_ids and accession_ids	            74	    ✅ Likely true biological conservation — keep all
"""
dna_dups = dna_df[dna_df.duplicated(subset=['dna'], keep=False)].sort_values(['dna'])
dup_summary = classify_dup_groups(dna_dups)

case1 = dup_summary[dup_summary['case'] == 'Case 1']
case2 = dup_summary[dup_summary['case'] == 'Case 2']
case3 = dup_summary[dup_summary['case'] == 'Case 3']
print(f"Case 1: {case1.shape[0]} sequences")
print(f"Case 2: {case2.shape[0]} sequences")
print(f"Case 3: {case3.shape[0]} sequences")

case2[:3]

Case 1: 0 sequences
Case 2: 1524 sequences
Case 3: 74 sequences


,dna,num_dups,files,assembly_ids,accession_ids,prefixes,case
0,AAAAAAAAAAAAAAAAAAAAAAAAAAAGTAGTAAACACACTCATAT...,2,"[GCA_018595275.1.qual.gto, GCF_018595275.1.qua...",[018595275.1],"[MW039262.1, NC_078362.1]","[GCA, GCF]",Case 2
1,AAAACAAAAAGAGATGGATCAAGACGTAGATCAAATTCTAAATAGA...,2,"[GCA_029888265.1.qual.gto, GCF_029888265.1.qua...",[029888265.1],"[NC_079031.1, OL774846.1]","[GCA, GCF]",Case 2
2,AAAACAGTAGTGTACCGCTGTGTATAAATAACCTAAGTGATAAAAG...,2,"[GCF_002831345.1.qual.gto, GCA_002831345.1.qua...",[002831345.1],"[NC_038733.1, KP792662.1]","[GCA, GCF]",Case 2


In [ ]:
# Explore duplicate groups with more than 2 files
dna_dups = dna_df[dna_df.duplicated(subset=['dna'], keep=False)].sort_values(['dna'])
dup_counts = dna_dups.groupby('dna')['file'].nunique().reset_index(name='num_files')
multi_dups = dup_counts[dup_counts['num_files'] > 2] # > 2 dups
multi_dup_records = dna_dups.merge(multi_dups[['dna']], on='dna') # Merge back to get full rows for inspection
multi_dup_records = multi_dup_records.sort_values('dna')
print(multi_dup_records.shape)

(24, 12)


# Protein (feature) data

In [24]:
# def get_protein_data_from_gto(gto_file_path):
#     """
#     Extract protein data and metadata from GTO file and return as a DataFrame.
#     """
#     # gto_file_path = quality_gto_dir / 'GCF_031497195.1.qual.gto'

#     ## Load GTO file
#     with open(gto_file_path, 'r') as file:
#         gto = json.load(file)

#     # # Build segment_id → replicon_type map
#     # segment_map = {c['id']: c.get('replicon_type', 'Unknown') for c in gto.get('contigs', [])}

#     ## 'features' key in GTO
#     # Each item in 'features' is a dict with keys: 'id', 'type', 'function', etc.
#     # fea is a list of dicts, each representing a 'features' item
#     fea = []
#     for d in gto['features']:
#         fea.append({k: d[k] for k in sorted(d.keys())})

#     # Aggregate the 'features' items into a DataFrame
#     df = []
#     # features_columns contains data items available in 'features' key
#     features_columns = ['id', 'type', 'function', 'protein_translation'] # is 'id' an Accession Number??
#     for i, item in enumerate(fea):
#         df.append([item[f] if f in item else np.nan for f in features_columns])
#         # df.append([item[f] for f in features_columns])
#     df = pd.DataFrame(df, columns=features_columns)
#     df['length'] = df['protein_translation'].apply(lambda x: len(x) if isinstance(x, str) else 0)
#     df['length'] = len(d['protein_translation']) if 'protein_translation' in d else 0

#     ## Add general metadata from the GTO file to the DataFrame
#     assembly_prefix, assembly_id = extract_assembly_info(gto_file_path.name)
#     meta = {
#         'file': gto_file_path.name,
#         'assembly_prefix': assembly_prefix,
#         'assembly_id': assembly_id,        
#         'ncbi_taxonomy_id': gto['ncbi_taxonomy_id'],
#         'genetic_code': gto['genetic_code'],
#         'scientific_name': gto['scientific_name'],
#         'quality': gto['quality']['genome_quality'],
#     }
#     meta_df = pd.DataFrame([meta])
#     df = pd.merge(meta_df, df, how='cross')

#     return df

In [ ]:
def get_protein_data_from_gto(gto_file_path):
    """
    Extract protein data and metadata from GTO file and return as a DataFrame.
    """
    # gto_file_path = quality_gto_dir / 'GCF_031497195.1.qual.gto'

    # Load GTO file
    with open(gto_file_path, 'r') as file:
        gto = json.load(file)

    # Build segment_id → replicon_type mapping
    # Iterates over 'contigs' key in GTO (if available)
    # Each item in 'contigs' is a dict with keys: 'id', 'dna', 'replicon_type', 'replicon_geometry'
    # 'id': segment id (e.g., NC_086346.1)
    # 'replicon_type': segment type/name (e.g., L, M, S)
    segment_map = {c['id']: c.get('replicon_type', 'Unknown') for c in gto.get('contigs', [])}

    ## Extract data from 'features'
    # features_columns contains data items available in 'features' key
    features_columns = ['id', 'type', 'function', 'protein_translation', 'location']
    rows = []
    for fea_dict in gto['features']: # d is a 'features' dict
        row = {k: fea_dict.get(k, None) for k in features_columns}
        row['length'] = len(fea_dict['protein_translation']) if 'protein_translation' in fea_dict else 0
        # Segment ID from location field
        if isinstance(fea_dict.get('location'), list) and len(fea_dict['location']) > 0:
            segment_id = fea_dict['location'][0][0]
        else:
            segment_id = None
        row['segment_id'] = segment_id
        row['replicon_type'] = segment_map.get(segment_id, 'Unknown')
        rows.append(row)

    df = pd.DataFrame(rows)

    ## Extract additional metadata from GTO file
    assembly_prefix, assembly_id = extract_assembly_info(gto_file_path.name)
    meta = {
        'file': gto_file_path.name,
        'assembly_prefix': assembly_prefix,
        'assembly_id': assembly_id,        
        'ncbi_taxonomy_id': gto['ncbi_taxonomy_id'],
        'genetic_code': gto['genetic_code'],
        'scientific_name': gto['scientific_name'],
        'quality': gto['quality']['genome_quality'],
    }
    meta_df = pd.DataFrame([meta])
    df = pd.merge(meta_df, df, how='cross')
    df.rename(columns={'protein_translation': 'prot_seq'}, inplace=True)

    return df

In [26]:
df = get_protein_data_from_gto(gto_file_path=quality_gto_dir / f'{ex_file}.qual.gto')
df

,file,assembly_prefix,assembly_id,ncbi_taxonomy_id,genetic_code,scientific_name,quality,id,type,function,prot_seq,location,length,segment_id,replicon_type
0,GCF_031497195.1.qual.gto,GCF,031497195.1,426786,11,Frijoles virus,Good,fig|426786.38.CDS.1,CDS,Nucleocapsid protein (N protein),MTDYADIAIAFAGEPINNAEVMGWVNEFAYEGFNAQRIIQLVQEKG...,"[[NC_086346.1, 70, +, 738]]",246,NC_086346.1,Small RNA Segment
1,GCF_031497195.1.qual.gto,GCF,031497195.1,426786,11,Frijoles virus,Good,fig|426786.38.CDS.2,CDS,Small Nonstructural Protein NSs (NSs Protein),MEIPLESFRLAVDFRPRFSDFYSRGDFPFRWGVGTFNSRVEQETTK...,"[[NC_086346.1, 1697, -, 681]]",227,NC_086346.1,Small RNA Segment
2,GCF_031497195.1.qual.gto,GCF,031497195.1,426786,11,Frijoles virus,Good,fig|426786.38.CDS.3,CDS,Pre-glycoprotein polyprotein GP complex (GPC p...,MNSKNIFYLALLSCSEAALRIRGRKELGRSEVCFSDSSPQEGVLVY...,"[[NC_086347.1, 21, +, 4101]]",1367,NC_086347.1,Medium RNA Segment
3,GCF_031497195.1.qual.gto,GCF,031497195.1,426786,11,Frijoles virus,Good,fig|426786.38.CDS.4,CDS,RNA-dependent RNA polymerase (L protein),MDDIISKQVELADGFNRRNLEHYNEILLDTPIPEFSVSKEGSGIII...,"[[NC_086348.1, 19, +, 6252]]",2084,NC_086348.1,Large RNA Segment
4,GCF_031497195.1.qual.gto,GCF,031497195.1,426786,11,Frijoles virus,Good,fig|426786.38.mat_peptide.1,mat_peptide,Phenuiviridae mature nonstructural 78-kD protein,DMLEGVIEKLRADILKERSLSTRLDNELSILIENQRVSKELRDKAR...,"[[NC_086347.1, 639, +, 1941]]",647,NC_086347.1,Medium RNA Segment
5,GCF_031497195.1.qual.gto,GCF,031497195.1,426786,11,Frijoles virus,Good,fig|426786.38.mat_peptide.2,mat_peptide,Mature envelope glycoprotein Gn (Gn protein),CLKVDYGSSCKTMIHMLESDNYKFFQAHSQHLSIFEAFNQKVIDVS...,"[[NC_086347.1, 1044, +, 1536]]",512,NC_086347.1,Medium RNA Segment
6,GCF_031497195.1.qual.gto,GCF,031497195.1,426786,11,Frijoles virus,Good,fig|426786.38.mat_peptide.3,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),SENIIASSSISKCRSESGKDVCVLSGTVYLKAGTIGSESCIIVKGT...,"[[NC_086347.1, 2601, +, 1518]]",506,NC_086347.1,Medium RNA Segment


In [27]:
# Aggregate protein data from all GTO files
print(f"Total files: {len(sorted(quality_gto_dir.glob('*.qual.gto')))}")

protein_data_fname = 'protein_data_all.tsv'

if not (output_dir / protein_data_fname).exists():
    print(f'Creating {output_dir / protein_data_fname}')
    dfs = []    
    for i, fpath in enumerate(sorted(quality_gto_dir.glob('*.gto'))):
        df = get_protein_data_from_gto(gto_file_path=fpath)
        dfs.append(df)

    prot_df = pd.concat(dfs, axis=0).reset_index(drop=True)
    prot_df.to_csv(output_dir / protein_data_fname, sep='\t', index=False)

else:
    print(f'Loading {output_dir / protein_data_fname}')
    prot_df = pd.read_csv(output_dir / protein_data_fname, sep='\t')

print(prot_df.shape)
print(prot_df.columns.tolist())

Total files: 1572
Creating /nfs/lambda_stor_01/data/apartin/projects/cepi/viral-segmatch/notebooks/../data/bunya_processed/protein_data_all.tsv
(8400, 15)
['file', 'assembly_prefix', 'assembly_id', 'ncbi_taxonomy_id', 'genetic_code', 'scientific_name', 'quality', 'id', 'type', 'function', 'prot_seq', 'location', 'length', 'segment_id', 'replicon_type']


In [ ]:
# Save DataFrame with samples that don't have protein data
prot_df_no_seq = prot_df[prot_df['prot_seq'].isna()]
print(f'Protein with missing sequence {prot_df_no_seq.shape}')
prot_df_no_seq.to_csv(output_dir / 'protein_data_no_seq.tsv', sep='\t', index=False)

(184, 15)


In [29]:
# Check duplicates
dups = prot_df[prot_df.duplicated(subset=['prot_seq'], keep=False)].sort_values(['prot_seq']).reset_index(drop=True)
dups.to_csv(output_dir / 'protein_duplicates.tsv', sep='\t', index=False)
print(dups.shape)
dups[:6]

(6646, 15)


,file,assembly_prefix,assembly_id,ncbi_taxonomy_id,genetic_code,scientific_name,quality,id,type,function,prot_seq,location,length,segment_id,replicon_type
0,GCF_000871205.1.qual.gto,GCF,000871205.1,70566,11,Akabane virus,Good,fig|70566.1857.mat_peptide.3,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ACIQEKEIETIDDAAMCIGLYQNITQPKEYNTFVKELSSTLSSHEI...,"[[NC_009895.1, 1418, +, 2808]]",936,NC_009895.1,Medium RNA Segment
1,GCA_000871205.1.qual.gto,GCA,000871205.1,70566,11,Akabane virus,Good,fig|70566.1858.mat_peptide.3,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ACIQEKEIETIDDAAMCIGLYQNITQPKEYNTFVKELSSTLSSHEI...,"[[AB100604.1, 1418, +, 2808]]",936,AB100604.1,Medium RNA Segment
2,GCF_002925575.1.qual.gto,GCF,002925575.1,1145238,11,Necocli virus,Poor,fig|1145238.48.mat_peptide.2,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ADTPILEPGWSDTAHGVGEIPMKTDLELDFSLPSSSSYSYRRKLTN...,"[[NC_043408.1, 1966, +, 1434]]",478,NC_043408.1,Medium RNA Segment
3,GCA_002925575.1.qual.gto,GCA,002925575.1,1145238,11,Necocli virus,Poor,fig|1145238.49.mat_peptide.2,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ADTPILEPGWSDTAHGVGEIPMKTDLELDFSLPSSSSYSYRRKLTN...,"[[KF494345.1, 1966, +, 1434]]",478,KF494345.1,Medium RNA Segment
4,GCA_002117715.1.qual.gto,GCA,002117715.1,3052486,11,Orthohantavirus montanoense,Good,fig|3052486.14.mat_peptide.2,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ADTPLAEVGWSDTAHGVGVIPLKTDLELDFALASSASYSYRRKLSN...,"[[AB620101.1, 1979, +, 1428]]",476,AB620101.1,Medium RNA Segment
5,GCF_002117715.1.qual.gto,GCF,002117715.1,3052486,11,Orthohantavirus montanoense,Good,fig|3052486.13.mat_peptide.2,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ADTPLAEVGWSDTAHGVGVIPLKTDLELDFALASSASYSYRRKLSN...,"[[NC_034397.1, 1979, +, 1428]]",476,NC_034397.1,Medium RNA Segment


# Protein data EDA

In [ ]:
# df.groupby('dna') retrieves the rows with the same dna sequence (i.e., dna sequence duplicates)
prot_dups = prot_df[prot_df.duplicated(subset=['prot_seq'], keep=False)].sort_values(['prot_seq'])
grouped = prot_dups.groupby('prot_seq')

# Explore the first group
for seq, grp in grouped:
    print(f"Sequence: {seq[:50]}")  # print a slice of seq
    print("Group (rows with this seq):")
    display(grp)
    break  # remove this to loop over more groups

Sequence: ACIQEKEIETIDDAAMCIGLYQNITQPKEYNTFVKELSSTLSSHEIEFLL
Group (rows with this seq):


,file,assembly_prefix,assembly_id,ncbi_taxonomy_id,genetic_code,scientific_name,quality,id,type,function,prot_seq,location,length,segment_id,replicon_type
5890,GCF_000871205.1.qual.gto,GCF,000871205.1,70566,11,Akabane virus,Good,fig|70566.1857.mat_peptide.3,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ACIQEKEIETIDDAAMCIGLYQNITQPKEYNTFVKELSSTLSSHEI...,"[[NC_009895.1, 1418, +, 2808]]",936,NC_009895.1,Medium RNA Segment
250,GCA_000871205.1.qual.gto,GCA,000871205.1,70566,11,Akabane virus,Good,fig|70566.1858.mat_peptide.3,mat_peptide,Mature envelope glycoprotein Gc (Gc protein),ACIQEKEIETIDDAAMCIGLYQNITQPKEYNTFVKELSSTLSSHEI...,"[[AB100604.1, 1418, +, 2808]]",936,AB100604.1,Medium RNA Segment


In [54]:
# Count exact duplicate protein sequences
prot_dups = prot_df[prot_df.duplicated(subset=['prot_seq'], keep=False)].sort_values(['prot_seq'])
print(prot_dups.shape)

# Count how many unique protein sequences are duplicated
dup_stats = prot_dups.groupby('prot_seq').agg(
        num_occurrences=('prot_seq', 'count'),
        num_files=('file', 'nunique'),
        functions=('function', lambda x: list(set(x))),
        replicons=('replicon_type', lambda x: list(set(x))),
    ).sort_values('num_occurrences', ascending=False).reset_index()

# TODO Should't num_occurrences and num_files be the same? If not, it means that the same protein seq appears multiple times in the same file.
print(dup_stats.shape)
display(dup_stats[:3])

(6646, 15)
(2852, 5)


,prot_seq,num_occurrences,num_files,functions,replicons
0,MSLSKCSNVDLKSVAMNANTVRLEPSLGEYPTLRRDLVECSCSVLT...,86,43,"[Nucleocapsid protein (N protein), Small Nonst...",[Small RNA Segment]
1,MSLSKCSNVDLKSVAMNANTVRLEPSLGEYPTLRRDLVECSCSVLT...,60,30,"[Nucleocapsid protein (N protein), Small Nonst...",[Small RNA Segment]
2,MSLSKCSNVDLKSVAMNANTVRLEPSLGEYPTLRRDLVECSCSVLT...,32,16,"[Nucleocapsid protein (N protein), Small Nonst...",[Small RNA Segment]


In [55]:
seq_name = dup_stats.iloc[0, :]['prot_seq']
df = prot_df[prot_df['prot_seq'].isin([seq_name])]
print(df.shape)
df[:3]

(86, 15)


,file,assembly_prefix,assembly_id,ncbi_taxonomy_id,genetic_code,scientific_name,quality,id,type,function,prot_seq,location,length,segment_id,replicon_type
1336,GCA_003087875.1.qual.gto,GCA,003087875.1,1316165,11,SFTS virus HNXY_115,Good,fig|1316165.15.CDS.3,CDS,Nucleocapsid protein (N protein),MSLSKCSNVDLKSVAMNANTVRLEPSLGEYPTLRRDLVECSCSVLT...,"[[KC292285.1, 29, +, 882]]",294,KC292285.1,Small RNA Segment
1337,GCA_003087875.1.qual.gto,GCA,003087875.1,1316165,11,SFTS virus HNXY_115,Good,fig|1316165.15.CDS.4,CDS,Small Nonstructural Protein NSs (NSs Protein),MSLSKCSNVDLKSVAMNANTVRLEPSLGEYPTLRRDLVECSCSVLT...,"[[KC292285.1, 29, +, 882]]",294,KC292285.1,Small RNA Segment
1348,GCA_003087915.1.qual.gto,GCA,003087915.1,1316166,11,SFTS virus HNXY_130,Good,fig|1316166.15.CDS.3,CDS,Nucleocapsid protein (N protein),MSLSKCSNVDLKSVAMNANTVRLEPSLGEYPTLRRDLVECSCSVLT...,"[[KC292289.1, 29, +, 882]]",294,KC292289.1,Small RNA Segment


In [34]:
# Step 1: Drop GCA entries if the same protein sequence exists in GCF for the same assembly_id

# Step 1.1: Identify all duplicated protein sequences
dups = prot_df[prot_df.duplicated(subset=['prot_seq'], keep=False)].copy()

# Step 1.2: Keep only rows where GCA and GCF exist for the same protein and same assembly_id
gca_gcf_pairs = dups.groupby(['prot_seq', 'assembly_id'])['assembly_prefix'].apply(set).reset_index()

# Filter to rows where both GCA and GCF versions exist
gca_gcf_pairs['has_both'] = gca_gcf_pairs['assembly_prefix'].apply(lambda x: 'GCA' in x and 'GCF' in x)
gca_gcf_with_both = gca_gcf_pairs[gca_gcf_pairs['has_both']]

# Step 1.3: Build a mask to drop the GCA rows from protein_df
drop_mask = prot_df.apply(
    lambda row: (
        row['assembly_prefix'] == 'GCA'
        and (row['prot_seq'], row['assembly_id']) in 
            set(zip(gca_gcf_with_both['prot_seq'], gca_gcf_with_both['assembly_id']))
    ),
    axis=1
)

# Step 1.4: Apply the filtering
protein_df_filtered = prot_df[~drop_mask].copy()

# Show how many rows were dropped
num_dropped = drop_mask.sum()
protein_df_filtered.shape, num_dropped


((5781, 15), 2619)

In [36]:
dups

,file,assembly_prefix,assembly_id,ncbi_taxonomy_id,genetic_code,scientific_name,quality,id,type,function,prot_seq,location,length,segment_id,replicon_type
0,GCA_000847345.1.qual.gto,GCA,000847345.1,11588,11,Rift Valley fever virus,Good,fig|11588.3927.CDS.1,CDS,RNA-dependent RNA polymerase (L protein),MDSILSKQLVDKTGFVRVPIKHFDCTMLTLALPTFDVSKMVDRITI...,"[[DQ375403.1, 19, +, 6279]]",2093,DQ375403.1,Large RNA Segment
1,GCA_000847345.1.qual.gto,GCA,000847345.1,11588,11,Rift Valley fever virus,Good,fig|11588.3927.CDS.2,CDS,Pre-glycoprotein polyprotein GP complex (GPC p...,MYVLLTILISVLVCEAVIRVSLSSTREETCFGDSTNPEMIEGAWDS...,"[[DQ380206.1, 21, +, 3594]]",1198,DQ380206.1,Medium RNA Segment
2,GCA_000847345.1.qual.gto,GCA,000847345.1,11588,11,Rift Valley fever virus,Good,fig|11588.3927.CDS.3,CDS,Small Nonstructural Protein NSs (NSs Protein),MDYFPVISVDLQSGRRVVSVEYFRGDGPPRIPYSMVGPCCVFLMHH...,"[[DQ380151.1, 35, +, 798]]",266,DQ380151.1,Small RNA Segment
3,GCA_000847345.1.qual.gto,GCA,000847345.1,11588,11,Rift Valley fever virus,Good,fig|11588.3927.CDS.4,CDS,Nucleocapsid protein (N protein),MDNYQELAIQFAAQAVDRNEIEQWVREFAYQGFDARRVIELLKQYG...,"[[DQ380151.1, 1652, -, 738]]",246,DQ380151.1,Small RNA Segment
4,GCA_000847345.1.qual.gto,GCA,000847345.1,11588,11,Rift Valley fever virus,Good,fig|11588.3927.mat_peptide.1,mat_peptide,Phenuiviridae mature nonstructural 78-kD protein,MYVLLTILISVLVCEAVIRVSLSSTREETCFGDSTNPEMIEGAWDS...,"[[DQ380206.1, 21, +, 2064]]",688,DQ380206.1,Medium RNA Segment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8395,GCF_031497195.1.qual.gto,GCF,031497195.1,426786,11,Frijoles virus,Good,fig|426786.38.CDS.3,CDS,Pre-glycoprotein polyprotein GP complex (GPC p...,MNSKNIFYLALLSCSEAALRIRGRKELGRSEVCFSDSSPQEGVLVY...,"[[NC_086347.1, 21, +, 4101]]",1367,NC_086347.1,Medium RNA Segment
8396,GCF_031497195.1.qual.gto,GCF,031497195.1,426786,11,Frijoles virus,Good,fig|426786.38.CDS.4,CDS,RNA-dependent RNA polymerase (L protein),MDDIISKQVELADGFNRRNLEHYNEILLDTPIPEFSVSKEGSGIII...,"[[NC_086348.1, 19, +, 6252]]",2084,NC_086348.1,Large RNA Segment
8397,GCF_031497195.1.qual.gto,GCF,031497195.1,426786,11,Frijoles virus,Good,fig|426786.38.mat_peptide.1,mat_peptide,Phenuiviridae mature nonstructural 78-kD protein,DMLEGVIEKLRADILKERSLSTRLDNELSILIENQRVSKELRDKAR...,"[[NC_086347.1, 639, +, 1941]]",647,NC_086347.1,Medium RNA Segment
8398,GCF_031497195.1.qual.gto,GCF,031497195.1,426786,11,Frijoles virus,Good,fig|426786.38.mat_peptide.2,mat_peptide,Mature envelope glycoprotein Gn (Gn protein),CLKVDYGSSCKTMIHMLESDNYKFFQAHSQHLSIFEAFNQKVIDVS...,"[[NC_086347.1, 1044, +, 1536]]",512,NC_086347.1,Medium RNA Segment
